In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from dotenv import load_dotenv
import json
from collections import defaultdict
import importlib
import warnings
warnings.filterwarnings('ignore')
import utils
importlib.reload(utils)
import os
from scipy.stats import linregress

# Load observations

In [2]:
# Load datasets for observations
def load_observations(file_path,sheet_name, cultivar_col,cultivar_name,treatment_col,treatment_name,location,max_observed_buds):
    # Define the expected column names for the final concatenated DataFrame
    TREATMENT = "Treatment"
    LOCATION = "Location"
    xls = pd.ExcelFile(file_path, engine="openpyxl")
    raw_df = pd.read_excel(xls, sheet_name = sheet_name)

    # Rename the Dates to Days of Year (DOY)
    doy_column = [pd.to_datetime(col).dayofyear for col in raw_df.columns if utils.is_date_column(col)]
    df = raw_df.rename(columns={col:pd.to_datetime(col).dayofyear for col in raw_df.columns if utils.is_date_column(col)})

    # Apply filtering for the given cultivar & treatments
    filtered_df = df[df[cultivar_col].isin([cultivar_name]) & df[treatment_col].isin([treatment_name])]

    # Rename columns
    columns_to_select = [treatment_col] + doy_column  # Ensure it's a flat list
    filtered_df = filtered_df[columns_to_select]
    filtered_df.rename(columns={treatment_col: TREATMENT}, inplace=True)
    # Add the LOCATION column to the DataFrame
    filtered_df[LOCATION] = location

    # Remove rows with any NaN values
    filtered_df = filtered_df.dropna()

    # Reset index 
    filtered_df.reset_index(drop=True)

    PBB, BB, BudBurstDOY = utils.BB_specifications(filtered_df[doy_column], max_observed_buds)
    return PBB, BB, BudBurstDOY
    


# WangEngel & GDH & Chill models

In [3]:
def Chill(Tc,T):
    return float(((4*T)/(Tc**2))*(Tc-T))

def Wangengel(MinTemp,OptTemp,MaxTemp,RefTemp,T):
    RelEff = 0.0
    RelEffRefTemp = 1.0
    p = 0.0

    if (T > MinTemp) and (T < MaxTemp):
        p = np.log(2.0) / np.log((MaxTemp - MinTemp) / (OptTemp - MinTemp))
        RelEff = (2 * np.pow(T - MinTemp, p) * np.pow(OptTemp - MinTemp, p) - np.pow(T - MinTemp, 2 * p)) / np.pow(OptTemp - MinTemp, 2 * p)

    if (RefTemp > MinTemp) and (RefTemp < MaxTemp):
        p = np.log(2.0) / np.log((MaxTemp - MinTemp) / (OptTemp - MinTemp))
        RelEffRefTemp = (2 * np.pow(RefTemp - MinTemp, p) * np.pow(OptTemp - MinTemp, p) - np.pow(RefTemp - MinTemp, 2 * p)) / np.pow(OptTemp - MinTemp, 2 * p)

    return float(RelEff / RelEffRefTemp)

def GDH(Tb,Tu,Tc,T):
    HU_values = np.zeros_like(T)
    mask1 = (T > Tb) & (T <= Tu)
    mask2 = (T > Tu) & (T <= Tc)

    HU_values[mask1] = 0.5 * (1 + np.cos(np.pi + np.pi * ((T[mask1] - Tb) / (Tu - Tb))))
    HU_values[mask2] =  (1 + np.cos(0.5*np.pi + 0.5*np.pi * ((T[mask2] - Tu) / (Tc - Tu))))

    return HU_values


# Chill/Heat Accumulation

In [4]:
# Chill & heat accumulation
def chill_heat_accumulation(start_doy, buds_targets, predicted_endo_release_doy, predicted_budbreak_doy, weather_since_start_doy, sheet_name, treatment_name):


    for bud_position,target in buds_targets.items():
        
        chill_requirement = target['chill']
        heat_requirement = target['heat']
        
        # Chill model parameter
        Tc = 18
        chillVal = 0
        for idx_endodormancy_release, temp in enumerate(weather_since_start_doy['temp']):
            chillVal += Chill(Tc, temp)
            if chillVal >= chill_requirement:
                predicted_endo_release_doy[sheet_name+treatment_name][bud_position] = int(weather_since_start_doy['doy'].iloc[idx_endodormancy_release])  # Get the date at this index
                break
        
        # from the date of endodormancy release, start accumulating heat
        heatVal = 0
        # wangengel model parameters
        MinTemp = 7
        OptTemp = 20
        MaxTemp = 25
        RefTemp = OptTemp # RefTemp = OptTemp scales the function to 0-1
        for idx_bb, temp in enumerate(weather_since_start_doy['temp'].iloc[idx_endodormancy_release::]):
            heatVal += Wangengel(MinTemp,OptTemp,MaxTemp,RefTemp,temp)
            if heatVal >= heat_requirement:
                predicted_budbreak_doy[sheet_name+treatment_name][bud_position] = int(weather_since_start_doy['doy'].iloc[idx_endodormancy_release+idx_bb+1])  # Get the date at this index
                break
    return predicted_endo_release_doy, predicted_budbreak_doy


In [5]:
with open("config_MultiDataFiles.json", "r") as file:
    config = json.load(file)

start_doy = pd.to_datetime('2016-03-01').dayofyear # 1st of March to start accumulation of chill units (year selection does not matter here)
chill_requirement_fixed = 900
heat_requirement_fixed = 500
buds_targets = {1:{'chill':chill_requirement_fixed, 'heat':heat_requirement_fixed}}#, 2:{'chill':chill_requirement_fixed, 'heat':200}, 3:{'chill':chill_requirement_fixed, 'heat':300}, 4:{'chill':chill_requirement_fixed, 'heat':400}, 5:{'chill':chill_requirement_fixed, 'heat':500}, 6:{'chill':chill_requirement_fixed, 'heat':600}}


predicted_endo_release_doy = defaultdict(lambda: dict())
predicted_budbreak_doy = defaultdict(lambda: dict())
observed_budbreak_doy = defaultdict(lambda: dict())

# Load environment variables
load_dotenv()
FOLDER_PATH = os.getenv('FOLDER_PATH')
for file_name, file_info in config.items():
        file_path = os.path.join(FOLDER_PATH, file_name)
        # Process each sheet
        for sheet_info in file_info['sheets']:
            # Extract sheet names, cultivars, and treatments
            sheet_name = sheet_info['sheet_name']
            location = sheet_info['location'].replace(' ','')
            max_observed_buds = sheet_info['max_observed_buds']
            cultivar_col, cultivar_name = list(sheet_info['cultivar'].items())[0]
            treatment_col, treatment_name = list(sheet_info['treatments'].items())[0]
            # Load the "Hourly" weather data
            weather_path = sheet_info['weather_path']
            weather_data = pd.read_csv(weather_path)
            weather_data = weather_data[['Date', 'Time', 'Air Temp. (°C)']].rename(columns={'Date': 'date', 'Time':'Time', 'Air Temp. (°C)':'temp'}, inplace=False)
            weather_data.insert(1, 'doy', [d.dayofyear for d in pd.to_datetime(weather_data['date'], format="mixed")])
            # Interpolate missing values
            # Convert '-' to NaN
            weather_data['temp'].replace('-', np.nan, inplace=True)
            weather_data['temp']=weather_data['temp'].apply(pd.to_numeric, errors='coerce')

            # Interpolate missing values
            weather_data['temp'].interpolate(method='linear', inplace=True)

            # start from the start_doy to accumulate chill/heat units
            weather_since_start_doy = weather_data[weather_data['doy']>start_doy]
            # print(weather_since_start_doy.head(30))

            # Predict the endodormancy release doy and budbreak doy
            predicted_endo_release_doy, predicted_budbreak_doy = chill_heat_accumulation(start_doy,buds_targets, predicted_endo_release_doy, predicted_budbreak_doy, weather_since_start_doy, sheet_name, treatment_name)

            # Load and calculate the observed budburst doy
            _, _, BudBurstDOY = load_observations(file_path,sheet_name, cultivar_col,cultivar_name,treatment_col,treatment_name,location,max_observed_buds)
            observed_budbreak_doy[sheet_name+treatment_name][1] = BudBurstDOY
            

In [19]:
all_bb_days = []
observed_all = []
predicted_all = []
fig = go.Figure()
for (trtmnt1,bb1),(trtmnt2,bb2) in zip(predicted_budbreak_doy.items(), observed_budbreak_doy.items()):
    predicted_bb= list(bb1.values())[0]
    observed_bb = list(bb2.values())[0]
    all_bb_days.append(observed_bb)
    all_bb_days.append(predicted_bb)
    observed_all.append(observed_bb)
    predicted_all.append(predicted_bb)
    fig.add_trace(go.Scatter(x=[observed_bb], y=[predicted_bb], mode='markers', marker=dict(size=15), name=trtmnt2))

# Add 1:1 dashed line
fig.add_trace(go.Scatter(x=[min(all_bb_days), max(all_bb_days)], y=[min(all_bb_days), max(all_bb_days)], mode='lines', line=dict(color="white", dash="dash"), name="1:1 Line"))  # Dashed black line

# Convert to numpy arrays
observed_all = np.array(observed_all)
predicted_all = np.array(predicted_all)
# Fit linear regression
slope, intercept, r_value, p_value, std_err = linregress(observed_all, predicted_all)
regression_line = slope * observed_all + intercept
# Calculate R²
r_squared = r_value ** 2
# Plot regression line
fig.add_trace(go.Scatter(x=observed_all, y=regression_line, mode='lines', line=dict(color="white", dash="solid"), name=f"Regression (y={slope:.2f}x + {intercept:.2f}), R2={round(r_squared,3)}"))

# Update layout
fig.update_layout(
    title="BudBurst",
    xaxis_title="Observed BudBurst DOY",
    yaxis_title="Predicted BudBurst DOY",
    template="plotly_dark")
fig.show()


In [33]:
##################################################
# If the predictions from APSIM is also available:
apsim_bb_predictions = [267,267,285,285,263,267,267,259,259,268,268,252,270]

# Fit linear regression for APSIM & Python predictions
_, _, r_real_hourly, _, _ = linregress(observed_all, predicted_all)
_, _, r_interpolated_hourly, _, _ = linregress(observed_all, apsim_bb_predictions)
# Calculate R²
r_squared_real_hourly = r_real_hourly ** 2
r_squared_interpolated_hourly = r_interpolated_hourly ** 2


apsim_python_comparison = pd.DataFrame({'Observed': observed_all, 'APSIM': apsim_bb_predictions, 'Python': predicted_all})

fig = px.scatter(apsim_python_comparison, x="Observed", y=["Python","APSIM"])
# Add the 1:1 line for comparison
fig.add_trace(go.Scatter(x=[min(all_bb_days), max(all_bb_days)], y=[min(all_bb_days), max(all_bb_days)], mode='lines', line=dict(color="black", dash="dash"), name="1:1 Line"))  # Dashed black line


fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(
    title="APSIM & Python Predictions (interpolated hourly temp. vs real hourly temp.)",
    xaxis_title="Observed BudBurst DOY",
    yaxis_title="Predicted BudBurst DOY")
fig.update_xaxes(
            tickmode="array", 
            tickvals=np.arange(observed_all.min(),observed_all.max(),1),  # Use all available dates as ticks
            tickangle=45,
            tickfont=dict(size=12))  # Rotate for better visibility

# Update legend labels
fig.for_each_trace(lambda t: t.update(name=f"{t.name} (R²={r_squared_real_hourly:.3f})") if t.name == "Python" else t.update(name=f"{t.name} (R²={r_squared_interpolated_hourly:.3f})"))


fig.show()
